In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

import os
import cv2
from tqdm import tqdm

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')



import sys
sys.path.append("../")
from per_segment_anything import sam_model_registry, SamPredictor

In [2]:
from IPython.display import display, Image

In [3]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.4])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)) 

In [4]:
ref_folder_path = "ref"
images_folder_path = "images"
output_path = 'outputs'

In [5]:
image_path = os.path.join(ref_folder_path, "original.jpg")

ref_save_folder_path = ref_folder_path
os.makedirs(ref_save_folder_path, exist_ok=True)


In [6]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import cv2

def on_click_figure(trace, points, state):
    x, y = points.point_inds[0]
    print(x, y)


# image = cv2.imread(image_path)
# cv2.imwrite(os.path.join(ref_save_folder_path, "original.jpg"), image)
# img_arr = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

sz = 512
img_arr = np.zeros((sz, sz), dtype=np.uint8)
for i in range(sz):
    for j in range(sz):
        img_arr[i, j] = int(255 * (i + j) / (2 * sz))
img_arr = np.stack([img_arr] * 3, axis=2)

img_trace = px.imshow(img_arr)
img_fig = go.FigureWidget()
img_fig.add_trace(img_trace.data[0])
img_fig.data[0].on_click(on_click_figure)
display(img_fig)

FigureWidget({
    'data': [{'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: [%{z[0]}, %{z[1]}, %{z[2]}]<extra></extra>',
              'name': '0',
              'source': ('data:image/png;base64,iVBORw0K' ... 'GdBAEqwO+5mwAAAABJRU5ErkJggg=='),
              'type': 'image',
              'uid': 'ae0beb8f-2004-45b0-a837-63c1ae341db2',
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'template': '...'}
})

238 213
187 187
110 119
284 119
247 398
182 352
312 261
284 179
